### 1. 모듈

In [1]:
import sys
sys.path.append("/usr/local/lib/python3.8/site-packages")

In [2]:
from selenium import webdriver
from random import randint
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from tqdm import tqdm
import time
import pandas as pd
import re

### 2. url 접속

In [36]:
driver = webdriver.Chrome('/Users/harampark/Documents/chromedriver')
url = 'http://likms.assembly.go.kr/bill/main.do'
driver.get(url)

In [37]:
search = driver.find_element_by_css_selector('li.gnbL2')
search.click()
time.sleep(2)


# driver.find_element_by_css_selector('#AGE_AREA > select:nth-child(2) > option:nth-child(2)').click()
# driver.find_element_by_css_selector('#AGE_AREA > select:nth-child(3) > option:nth-child(2)').click()

search = driver.find_element_by_css_selector('div.mt20.alignC > button.btnd')
search .click()
time.sleep(2)

more_bill = driver.find_element_by_css_selector('#pageSizeOption > option:nth-child(4)').click()

### 3. 크롤링

In [46]:
num_list = []
handled_list = []
title_list = []
proposer_type_list = []
propose_date_list = []
resolution_date_list = []
resolution_result_list = []
progress_list = []

proposer_list = []
paper_list = []
session_list = []
reason_list = []
board_list = []

In [47]:
n_times = 0

## 원하는 시작페이지
n_iter = 3

# driver.find_element_by_css_selector('#pageListViewArea > a:nth-child(11)').click()
# while n_times <= n_iter - 2:
    #driver.find_element_by_css_selector('#pageListViewArea > a:nth-child(13)').click()
    #n_times +=1

In [ ]:
if n_iter == 0:
    start = 1; end = 11
else:
    start = 3; end = 13
    
## 10페이지가 끝나면 다음 10페이지를 불러옵니다.
while n_iter <= 50:

    ## start번째 페이지에 접속합니다.
    for k in tqdm(range(start,end)):
        
        driver.find_element_by_css_selector(f'#pageListViewArea > a:nth-child({k})').click()
        ## start번째 페이지의 i번째 의안에 접근합니다.
        for i in range(1,101):
            ## i 번째 의안에 접근합니다.
            MAX_SLEEP_TIME = 3
            rand_value = randint(1, MAX_SLEEP_TIME)
            time.sleep(rand_value)
            bill = driver.find_element_by_css_selector(f'body > div > div.contentWrap > div.subContents > div > div.tableCol01 > table > tbody > tr:nth-child({i})')

            num = bill.find_element_by_css_selector('td:nth-child(1)').text
            title = bill.find_element_by_css_selector('td.alignL').text
            proposer_type = bill.find_element_by_css_selector('td:nth-child(3)').text
            propose_date = bill.find_element_by_css_selector('td:nth-child(4)').text
            resolution_date = bill.find_element_by_css_selector('td:nth-child(5)').text
            resolution_result = bill.find_element_by_css_selector('td:nth-child(6)').text
            progress = bill.find_element_by_css_selector('td:nth-child(8)').text

            num_list.append(num)
            if resolution_date == '':
                handled_list.append('계')
            else:
                handled_list.append('처')
            title_list.append(title)
            proposer_type_list.append(proposer_type)
            propose_date_list.append(propose_date)
            resolution_date_list.append(resolution_date)
            resolution_result_list.append(resolution_result)
            progress_list.append(progress)


            ## start번째 페이지의 i번째 의안에 접속합니다.
            bill = driver.find_element_by_css_selector(f'body > div > div.contentWrap > div.subContents > div > div.tableCol01 > table > tbody > tr:nth-child({i}) > td.alignL > div.pl25 > a')
            bill.click()
            
            MAX_SLEEP_TIME = 3
            rand_value = randint(1, MAX_SLEEP_TIME)
            time.sleep(rand_value)
            paper = driver.find_element_by_css_selector('body > div > div.contentWrap > div.subContents > div > div.contIn > div.tableCol01 > table > tbody > tr > td:nth-child(4)').text
            session = driver.find_element_by_css_selector('body > div > div.contentWrap > div.subContents > div > div.contIn > div.tableCol01 > table > tbody > tr > td:nth-child(5)').text     

            paper_list.append(paper)
            session_list.append(session)
            
            ## 제안이유를 담습니다.
            try:
                # 더 보기를 클릭합니다.
                more_info = driver.find_element_by_css_selector('#summaryContentMoreBtn').click()
                reason_raw_text = driver.find_element_by_css_selector('div.contIn > div.textType02').text
                
                content = []
                reason_split = re.split('\r|\t|\n|\xa0', reason_raw_text)
                content.append(''.join(reason_split))
                reason_list.append(content)            
                
            except:
                reason_list.append([]) 

            ## 소관위원회를 담습니다.
            try:
                board = driver.find_element_by_css_selector('body > div > div.contentWrap > div.subContents > div > div:nth-child(5) > div > table > tbody > tr > td:nth-child(1)').text
                board_list.append(board)
            except:
                board_list.append([])
            
            ## 제안자를 담습니다.
            member_list = []    
            try:
                proposer = driver.find_element_by_css_selector('body > div > div.contentWrap > div.subContents > div > div.contIn > div.tableCol01 > table > tbody > tr > td:nth-child(3) > a > img')
                proposer.click()

                # 새로운 창으로 초점을 바꿉니다.
                driver.switch_to_window(driver.window_handles[1])
                driver.get_window_position(driver.window_handles[1])

                member_num = len(driver.find_elements_by_tag_name('a'))
                member_end = (member_num//3) + member_num + 2

                for j in range(2, member_end):
                    try:
                        member = driver.find_element_by_css_selector(f'#periodDiv > div.layerInScroll.coaTxtScroll > div > a:nth-child({j})').text
                        member_list.append(member)
                    except:
                        continue
                        
                driver.close()
                proposer_list.append(member_list)

                # 원래 창으로 초점을 바꿉니다.
                driver.switch_to_window(driver.window_handles[0])
                driver.get_window_position(driver.window_handles[0])

            except:
                proposer_list.append([])
    
            driver.back()
            driver.execute_script("window.scrollTo(0, window.scrollY + 70)")

    driver.find_element_by_css_selector(f'#pageListViewArea > a:nth-child({10+start})').click()
    if n_iter == 0:
        start = 1; end = 11
    else:
        start = 3; end = 13
    n_iter += 1
    MAX_SLEEP_TIME = 6
    rand_value = randint(4, MAX_SLEEP_TIME)
    time.sleep(rand_value)

  0%|          | 0/10 [00:00<?, ?it/s]

### 4. 데이터프레임

In [45]:
result = [num_list, handled_list, title_list, proposer_type_list, propose_date_list, resolution_date_list, resolution_result_list,\
          progress_list, paper_list, session_list, reason_list, board_list, proposer_list]
result_df = pd.DataFrame(result).T
result_df.columns = ['의안번호', '처리여부', '의안명', '제안자구분', '제안일자', '의결일자', '의결결과', \
                     '심사진행상태', '문서', '제안회기' , '제안이유', '소관위원회', '제안자']
result_df

,의안번호,처리여부,의안명,제안자구분,제안일자,의결일자,의결결과,심사진행상태,문서,제안회기,제안이유,소관위원회,제안자
0,2102714,계,초·중등교육법 일부개정법률안(서동용의원 등 13인),의원,2020-08-05,,,접수,의안원문,제21대 (2020~2024) 제380회,[제안이유 및 주요내용현행법에 따르면 학교운영의 자율성을 높이고 지역의 실정과 특성...,교육위원회,"[서동용(더불어민주당/徐東榕), 고영인(더불어민주당/高永寅), 김정호(더불어민주당/..."
1,2102713,계,사립학교법 일부개정법률안(서동용의원 등 13인),의원,2020-08-05,,,접수,의안원문,제21대 (2020~2024) 제380회,[제안이유 및 주요내용「초ㆍ중등교육법」에 따르면 학교운영의 자율성을 높이고 지역의 ...,교육위원회,"[서동용(더불어민주당/徐東榕), 고영인(더불어민주당/高永寅), 김정호(더불어민주당/..."
2,2102712,계,민간임대주택에 관한 특별법 일부개정법률안(서동용의원 등 11인),의원,2020-08-05,,,소관위접수,의안원문,제21대 (2020~2024) 제380회,[제안이유 및 주요내용현행법은 공공주택사업자와 임차인대표회의 사이에서 임대료의 증액...,국토교통위원회,"[서동용(더불어민주당/徐東榕), 강민정(열린민주당/姜旼姃), 고영인(더불어민주당/高..."
3,2102711,계,비대면중소벤처기업 육성에 관한 법률안(정태호의원 등 13인),의원,2020-08-05,,,소관위접수,의안원문,제21대 (2020~2024) 제380회,[제안이유전세계적으로 디지털 및 비대면 경제가 새로운 경제 모델로 중요하게 부상되고...,산업통상자원중소벤처기업위원회,"[정태호(더불어민주당/鄭泰浩), 강준현(더불어민주당/康準鉉), 박상혁(더불어민주당/..."
4,2102710,계,아이돌봄 지원법 일부개정법률안(이용호의원 등 11인),의원,2020-08-05,,,소관위접수,의안원문,제21대 (2020~2024) 제380회,[제안이유 및 주요내용현행법은 일정한 자격과 기준을 갖춘 아이돌보미와 아이돌봄서비스...,여성가족위원회,"[이용호(무소속/李容鎬), 김영호(더불어민주당/金映豪), 김진표(더불어민주당/金振杓..."
5,2102709,계,산업재해보상보험법 일부개정법률안(박대수의원 등 11인),의원,2020-08-05,,,소관위접수,의안원문,제21대 (2020~2024) 제380회,"[제안이유 및 주요내용현행법은 업무상 사고, 업무상 질병 및 출퇴근 재해 등으로 인...",환경노동위원회,"[박대수(미래통합당/朴大壽), 강기윤(미래통합당/姜起潤), 김선교(미래통합당/金善敎..."
6,2102708,계,근로기준법 일부개정법률안(박대수의원 등 10인),의원,2020-08-05,,,소관위접수,의안원문,제21대 (2020~2024) 제380회,[제안이유 및 주요내용통계청에 따르면 우리나라 비정규직 근로자 비율은 36.4%(2...,환경노동위원회,"[박대수(미래통합당/朴大壽), 강기윤(미래통합당/姜起潤), 김성원(미래통합당/金成願..."
7,2102707,계,북한이탈주민의 보호 및 정착지원에 관한 법률 일부개정법률안(지성호의원 등 10인),의원,2020-08-05,,,소관위접수,의안원문,제21대 (2020~2024) 제380회,[제안이유 및 주요내용통일부는 북한이탈주민의 신변안전을 위하여 국방부장관 또는 경찰...,외교통일위원회,"[지성호(미래통합당/地成浩), 권영세(미래통합당/權寧世), 김석기(미래통합당/金碩基..."
8,2102706,계,지방재정법 일부개정법률안(김회재의원 등 10인),의원,2020-08-05,,,소관위접수,의안원문,제21대 (2020~2024) 제380회,"[제안이유 및 주요내용현행법은 공해, 소음 및 환경오염 등 외부불경제를 유발하는 시...",행정안전위원회,"[김회재(더불어민주당/金會在), 강준현(더불어민주당/康準鉉), 김민철(더불어민주당/..."
9,2102705,계,아동복지법 일부개정법률안(남인순의원 등 14인),의원,2020-08-05,,,소관위접수,의안원문,제21대 (2020~2024) 제380회,[제안이유 및 주요내용현행법에서는 보장원 또는 아동보호전문기관의 장은 아동학대가 종...,보건복지위원회,"[남인순(더불어민주당/南仁順), 강선우(더불어민주당/姜仙祐), 김경만(더불어민주당/..."


In [26]:
result_df.to_csv('21대 국회 의안정보(1).csv', encoding='utf-8-sig', index=False)